In [1]:
outputMetricFile = "result3epochsLSTM_oversample01_smote.csv"

# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import pandas as pd
# Bibliotecas Auxiliares
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

import datetime;
import warnings
warnings.filterwarnings("ignore")

print(tf.__version__)

2023-03-13 20:25:07.482271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 20:25:07.912306: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-13 20:25:08.957228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-13 20:25:08.959668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

2.10.0


In [2]:
# configs
EPOCHS = 5
BATCH_SIZE = 32

baseFolder = "../data_2019_processed/"

In [3]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    array = []
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    results['matrix'] = np.array(matrix,dtype=object)
    results['TP'] = matrix[0][0]
    results['FP'] = matrix[0][1]
    results['FN'] = matrix[1][0]
    results['TN'] = matrix[1][1]
    
    array.append(accuracy)
    array.append(precision)
    array.append(recall)
    array.append(f1)
    array.append(kappa)
    array.append(auc)
    array.append(np.array(matrix,dtype=object))
    array.append(matrix[0][0]) # TP
    array.append(matrix[0][1]) # FP
    array.append(matrix[1][0]) # FN
    array.append(matrix[1][1]) # TN
    
    return results, array

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
    return [accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score]
    
def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    for column in outputClasses:
        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset

In [4]:
X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
X_train_under = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample_nm.csv")
X_train_over = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_oversample_smote.csv")
#X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_undersample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")

#X_train, X_test = train_test_split(AA,test_size=0.25)
#X_train = pd.read_csv("train_temp.csv")
#X_test = pd.read_csv("test_temp.csv")

In [5]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407451 entries, 0 to 407450
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            407451 non-null  float64
 1   location            407451 non-null  float64
 2   timestamp           407451 non-null  float64
 3   time_to_next_alarm  407451 non-null  float64
 4   sound               407451 non-null  float64
 5   proximity           407451 non-null  float64
 6   phone_lock          407451 non-null  float64
 7   light               407451 non-null  float64
 8   day_of_week         407451 non-null  float64
 9   minutes_day         407451 non-null  float64
 10  timestamp_text      407451 non-null  object 
 11  class               407451 non-null  object 
dtypes: float64(10), object(2)
memory usage: 37.3+ MB
None


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.0,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.0,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.000165,0.0,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.001449,0.0,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.000869,0.981944,0.601849,1.0,0.0,0.000198,0.0,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
407446,0.25,1.0,0.006924,0.992956,0.644370,1.0,1.0,0.000000,0.0,0.549687,2018-06-12 13:11:39,awake
407447,0.25,1.0,0.006924,0.992956,0.644370,1.0,1.0,0.000000,0.0,0.550382,2018-06-12 13:12:09,awake
407448,0.25,1.0,0.006924,0.992857,0.624127,1.0,1.0,0.000538,0.0,0.551077,2018-06-12 13:13:37,awake
407449,0.00,1.0,0.006924,0.992758,0.540295,0.0,0.0,0.000000,0.0,0.551772,2018-06-12 13:14:07,awake


In [6]:
# create two classes based on the single class

# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)

# transform output to one_hot_encoding for the testing dataset
X_train_under = transform_output_nominal_class_into_one_hot_encoding(X_train_under)

# transform output to one_hot_encoding for the testing dataset
X_train_over = transform_output_nominal_class_into_one_hot_encoding(X_train_over)

In [7]:
# activity	location	timestamp	day_of_week	light	phone_lock	proximity	sound	time_to_next_alarm	minutes_day
inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound","time_to_next_alarm", "minutes_day"]
#inputFeatures = ["activity","location","day_of_week","light","phone_lock","proximity","sound", "minutes_day"]

outputClasses = ["awake","asleep"]



def create_keras_model(inputFeatures,outputClasses):
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(len(inputFeatures),)),
        tf.keras.layers.Dense(len(inputFeatures)),
        tf.keras.layers.Dense(12, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(12, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(len(outputClasses), activation='softmax')
    ])

In [8]:
print("--------------")
print("--------------")
print("Initializing raw test")
print("--------------")
print("Batch Size: ",BATCH_SIZE)
print("Epochs: ",EPOCHS)
print("Input/Features:",len(inputFeatures))
print("Outputs:",len(outputClasses))
print("--------------")

# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

# instanciates the model
model = create_keras_model(inputFeatures,outputClasses)

# compiles the model
model.compile(optimizer='adam',
             # loss='binary_crossentropy',loss='categorical_crossentropy',
              #loss='binary_crossentropy',  sparse_categorical_crossentropy         
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
# 
model.fit(X_train_data, y_train_data, epochs=EPOCHS,batch_size=BATCH_SIZE)

results = model.evaluate(X_test_data, y_test_data, return_dict=True,verbose=0)

print(results)

--------------
--------------
Initializing raw test
--------------
Batch Size:  32
Epochs:  5
Input/Features: 9
Outputs: 2
--------------


2023-03-13 20:25:16.888993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 20:25:16.970105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 20:25:16.970456: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 20:25:16.972523: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Epoch 1/5
   32/12733 [..............................] - ETA: 1:04 - loss: 0.7532 - categorical_accuracy: 0.4404

2023-03-13 20:25:20.376318: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


12733/12733 [==============================] - 77s 6ms/step - loss: 0.3945 - categorical_accuracy: 0.8319
Epoch 2/5
12733/12733 [==============================] - 77s 6ms/step - loss: 0.3758 - categorical_accuracy: 0.8414
Epoch 3/5
12733/12733 [==============================] - 77s 6ms/step - loss: 0.3714 - categorical_accuracy: 0.8439
Epoch 4/5
12733/12733 [==============================] - 77s 6ms/step - loss: 0.3685 - categorical_accuracy: 0.8449
Epoch 5/5
12733/12733 [==============================] - 75s 6ms/step - loss: 0.3669 - categorical_accuracy: 0.8450
{'loss': 0.3871196508407593, 'categorical_accuracy': 0.8360121250152588}


In [9]:
yhat_probs = model.predict(X_test_data)
# dataset
probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])

test = list()
print('')
print('awake')
res,resA = printMetrics(y_test_data['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res,resA = printMetrics(y_test_data['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4216/4216 [==============================] - 6s 1ms/step

awake
Accuracy: 0.836012
Precision: 0.868023
Recall: 0.941623
F1 score: 0.903327
Cohens kappa: 0.369169
ROC AUC: 0.787335
\Confusion Matrix
[[  9422  15713]
 [  6407 103346]]

asleep
Accuracy: 0.836012
Precision: 0.595237
Recall: 0.374856
F1 score: 0.460014
Cohens kappa: 0.369169
ROC AUC: 0.787335
\Confusion Matrix
[[103346   6407]
 [ 15713   9422]]

Global
2
accuracy:  0.8360120989265168
precision:  0.7316300040141837
recall:  0.6582396211950189
f1_score:  0.6816702095685439
cohen_kappa_score:  0.3691689354090413
roc_auc_score:  0.787335269266787


[0.8360120989265168,
 0.7316300040141837,
 0.6582396211950189,
 0.6816702095685439,
 0.3691689354090413,
 0.787335269266787]

In [10]:
print("--------------")
print("--------------")
print("Initializing Undersample test")
print("--------------")
print("Batch Size: ",BATCH_SIZE)
print("Epochs: ",EPOCHS)
print("Input/Features:",len(inputFeatures))
print("Outputs:",len(outputClasses))
print("--------------")

# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train_under,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train_under,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

# instanciates the model
model = create_keras_model(inputFeatures,outputClasses)

# compiles the model
model.compile(optimizer='adam',
             # loss='binary_crossentropy',loss='categorical_crossentropy',
              #loss='binary_crossentropy',  sparse_categorical_crossentropy         
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
# 
model.fit(X_train_data, y_train_data, epochs=EPOCHS,batch_size=BATCH_SIZE)

results = model.evaluate(X_test_data, y_test_data, return_dict=True,verbose=0)

print(results)

--------------
--------------
Initializing Undersample test
--------------
Batch Size:  32
Epochs:  5
Input/Features: 9
Outputs: 2
--------------
Epoch 1/5
4617/4617 [==============================] - 27s 6ms/step - loss: 0.6418 - categorical_accuracy: 0.6277
Epoch 2/5
4617/4617 [==============================] - 27s 6ms/step - loss: 0.6182 - categorical_accuracy: 0.6571
Epoch 3/5
4617/4617 [==============================] - 26s 6ms/step - loss: 0.6121 - categorical_accuracy: 0.6628
Epoch 4/5
4617/4617 [==============================] - 27s 6ms/step - loss: 0.6078 - categorical_accuracy: 0.6671
Epoch 5/5
4617/4617 [==============================] - 26s 6ms/step - loss: 0.6049 - categorical_accuracy: 0.6695
{'loss': 0.6557937264442444, 'categorical_accuracy': 0.6333476901054382}


In [11]:
yhat_probs = model.predict(X_test_data)
# dataset
probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])

test = list()
print('')
print('awake')
res,resA = printMetrics(y_test_data['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res,resA = printMetrics(y_test_data['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4216/4216 [==============================] - 5s 1ms/step

awake
Accuracy: 0.633348
Precision: 0.915662
Recall: 0.605113
F1 score: 0.728680
Cohens kappa: 0.230283
ROC AUC: 0.711646
\Confusion Matrix
[[19018  6117]
 [43340 66413]]

asleep
Accuracy: 0.633348
Precision: 0.304981
Recall: 0.756634
F1 score: 0.434732
Cohens kappa: 0.230283
ROC AUC: 0.711646
\Confusion Matrix
[[66413 43340]
 [ 6117 19018]]

Global
2
accuracy:  0.6333476662119685
precision:  0.6103217005658944
recall:  0.6808737376583984
f1_score:  0.5817060253302145
cohen_kappa_score:  0.23028255498747197
roc_auc_score:  0.7116458715077294


[0.6333476662119685,
 0.6103217005658944,
 0.6808737376583984,
 0.5817060253302145,
 0.23028255498747197,
 0.7116458715077294]

In [12]:
print("--------------")
print("--------------")
print("Initializing Oversample test")
print("--------------")
print("Batch Size: ",BATCH_SIZE)
print("Epochs: ",EPOCHS)
print("Input/Features:",len(inputFeatures))
print("Outputs:",len(outputClasses))
print("--------------")

# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train_over,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train_over,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

# instanciates the model
model = create_keras_model(inputFeatures,outputClasses)

# compiles the model
model.compile(optimizer='adam',
             # loss='binary_crossentropy',loss='categorical_crossentropy',
              #loss='binary_crossentropy',  sparse_categorical_crossentropy         
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
# 
model.fit(X_train_data, y_train_data, epochs=EPOCHS,batch_size=BATCH_SIZE)

results = model.evaluate(X_test_data, y_test_data, return_dict=True,verbose=0)

print(results)

--------------
--------------
Initializing Oversample test
--------------
Batch Size:  32
Epochs:  5
Input/Features: 9
Outputs: 2
--------------
Epoch 1/5
20850/20850 [==============================] - 119s 6ms/step - loss: 0.5512 - categorical_accuracy: 0.7283
Epoch 2/5
20850/20850 [==============================] - 121s 6ms/step - loss: 0.5309 - categorical_accuracy: 0.7426
Epoch 3/5
20850/20850 [==============================] - 121s 6ms/step - loss: 0.5250 - categorical_accuracy: 0.7458
Epoch 4/5
20850/20850 [==============================] - 121s 6ms/step - loss: 0.5208 - categorical_accuracy: 0.7485
Epoch 5/5
20850/20850 [==============================] - 120s 6ms/step - loss: 0.5179 - categorical_accuracy: 0.7501
{'loss': 0.47760361433029175, 'categorical_accuracy': 0.8022581934928894}


In [13]:
yhat_probs = model.predict(X_test_data)
# dataset
probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])

test = list()
print('')
print('awake')
res,resA = printMetrics(y_test_data['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res,resA = printMetrics(y_test_data['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4216/4216 [==============================] - 6s 1ms/step

awake
Accuracy: 0.802258
Precision: 0.922747
Recall: 0.826137
F1 score: 0.871774
Cohens kappa: 0.445597
ROC AUC: 0.799008
\Confusion Matrix
[[17544  7591]
 [19082 90671]]

asleep
Accuracy: 0.802258
Precision: 0.479004
Recall: 0.697991
F1 score: 0.568126
Cohens kappa: 0.445597
ROC AUC: 0.799008
\Confusion Matrix
[[90671 19082]
 [ 7591 17544]]

Global
2
accuracy:  0.802258169740822
precision:  0.700875667581794
recall:  0.7620638601935416
f1_score:  0.7199495931352058
cohen_kappa_score:  0.4455973116938615
roc_auc_score:  0.7990079662050198


[0.802258169740822,
 0.700875667581794,
 0.7620638601935416,
 0.7199495931352058,
 0.4455973116938615,
 0.7990079662050198]